# Exercise 5: Training YOLOv8n and Validating with Laptop Camera

## Objective
Train a YOLOv8 Nano model and validate it using the laptop camera.

We will:
1. Install YOLO (Ultralytics)
2. Train on reference dataset (COCO8) to verify setup
3. Validate using laptop camera with **Safety Interlocks**

## Step 1: Install Dependencies
We need the `ultralytics` package which contains the YOLO implementation.

In [ ]:
!pip install -q ultralytics opencv-python

## Step 2: Import Libraries

In [1]:
from ultralytics import YOLO
import cv2
import os

## Step 3: Reference Training (COCO8)

We use `yolo26n.pt` (Nano), which is small and fast for laptops.

In [9]:
# Load the model (will download automatically)
model = YOLO('yolo26n.pt')

# Train on the tiny COCO8 dataset just to verify the pipeline works
results = model.train(
    data='coco8.yaml',
    epochs=5,
    imgsz=640,
    project='runs/detect',  # Explicitly set save path
    name='train_coco'       # Name the experiment
)

Ultralytics 8.4.14  Python-3.12.8 torch-2.8.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco8.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train_coco, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0

## Step 4: Validate Using Laptop Camera (Local Only)

**WARNING:** This cell will fail if run in Google Colab or a Cloud Server (no webcam attached).
It requires a **Local Jupyter Notebook**.

In [36]:
# Load the model we just trained (or the base model if training failed)
model_path = 'best.pt'

if os.path.exists(model_path):
    print(f"Loading trained model: {model_path}")
    model = YOLO(model_path)
else:
    print("Trained weights not found.")    

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
else:
    print("Webcam opened. Press 'q' to quit.")
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break

            # Run inference
            results = model(frame, verbose=False)
            
            # Plot results on the frame
            annotated_frame = results[0].plot()

            # Display
            cv2.imshow('YOLO Live Detection', annotated_frame)

            # Break loop on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except KeyboardInterrupt:
        print("Interrupted by user!")
    finally:
        # ALWAYS release the camera, even if code crashes
        cap.release()
        cv2.destroyAllWindows()
        print("Camera released.")

Loading trained model: best.pt
Webcam opened. Press 'q' to quit.
Interrupted by user!
Camera released.
